 # 🔥🔥🔥 Cassava 🍂 Disease Classification - EDA + BL Model 🔥🔥🔥




# Introduction


Cassava Leaf Diseases in the dataset are 

There are 5 classifications :
* 0: [Cassava Bacterial Blight (CBB)](https://en.wikipedia.org/wiki/Bacterial_blight_of_cassava)
* 1: [Cassava Brown Streak Disease (CBSD)](https://en.wikipedia.org/wiki/Cassava_brown_streak_virus_disease)
* 2: [Cassava Green Mottle (CGM)](https://en.wikipedia.org/wiki/Cassava_green_mottle_virus)
* 3: [Cassava Mosaic Disease (CMD)](https://en.wikipedia.org/wiki/Cassava_mosaic_virus)
* 4: Healthy leaf



# 🙌 😊 👍 Upvote if you find this Kernal useful 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

from keras.utils import to_categorical, Sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop,Adam

In [ ]:
path = '/kaggle/input/cassava-leaf-disease-classification/'


In [ ]:
train_data = pd.read_csv(path+'train.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

In [ ]:
train_data.head()

# Exploratory Data Analysis

In [ ]:
print('number of train data:', len(train_data))
print('number of train images:', len(os.listdir(path+'train_images/')))
print('number of test images:', len(os.listdir(path+'test_images/')))

# Let see the distribution of labled data

In [ ]:
train_data['label'].hist(bins=4)

## Observation - For sure its an imbalance multi Class classification problem 

In [ ]:
!ls '/kaggle/input/cassava-leaf-disease-classification/train_images/'

In [ ]:
img = cv2.imread(path+'train_images/'+'2519147193.jpg')
plt.imshow(img)
plt.show()

In [ ]:
train_data[train_data['image_id']=='2519147193.jpg']

In [ ]:
train_data.info()

In [ ]:
class_label = {
    0: "Cassava Bacterial Blight (CBB)",
    1: "Cassava Brown Streak Disease (CBSD)",
    2: "Cassava Green Mottle (CGM)",
    3: "Cassava Mosaic Disease (CMD)",
    4: "Healthy"
}

In [ ]:
train_data['class_name'] = train_data['label'].map(class_label)

## Observation - Above leaf is infected with Cassava Green Mottle (CGM) disease 

In [ ]:
img = cv2.imread(path+'train_images/'+'1000201771.jpg')
plt.imshow(img)
plt.show()

In [ ]:
def visualize_batch(image_ids, labels):
    plt.figure(figsize=(16, 12))
    
    for ind, (image_id, label) in enumerate(zip(image_ids, labels)):
        plt.subplot(3, 3, ind + 1)
        image = cv2.imread(os.path.join(path, "train_images", image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(f"Class: {label}", fontsize=12)
        plt.axis("off")
    
    plt.show()

In [ ]:
tmp_df = train_data.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["class_name"].values

visualize_batch(image_ids, labels)

# Cassava Bacterial Blight (CBB) - Disease sample 

In [ ]:
tmp_df = train_data[train_data["label"] == 0]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

# Cassava Brown Streak Disease (CBSD) - Disease Sample

In [ ]:
tmp_df = train_data[train_data["label"] == 1]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

# Cassava Green Mottle (CGM) - Disease Sample

In [ ]:
tmp_df = train_data[train_data["label"] == 2]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

# Cassava Mosaic Disease (CMD) - Disease Sample 

In [ ]:
tmp_df = train_data[train_data["label"] == 3]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

## Observation - Above leaf is heathy one 

## Healthy Leaf images

In [ ]:
tmp_df = train_data[train_data["label"] == 4]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

## Hyperparams 

In [ ]:
batch_size = 3
img_size = 64
img_channel = 3
epochs = 5

In [ ]:
to_categorical(train_data['label']) ## Convert into One hot encoded vector of 1 * 4 

In [ ]:
y_train =to_categorical(train_data['label'])

## Class wights

In [ ]:
class_weight = dict(zip(range(0, 7), (train_data['label'].value_counts()/len(train_data))))

In [ ]:
train_data['label'].value_counts()

In [ ]:
dict(zip(range(0, 7), (train_data['label'].value_counts()/len(train_data))))

In [ ]:
class_weight = dict(zip(range(0, 7), (train_data['label'].value_counts()/len(train_data))))

## Thanks to https://www.kaggle.com/drcapa/cassava-leaf-disease-classification-starter-keras kernal  

# Defining Dataset Class for building model

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, path, list_IDs, labels, batch_size, img_size, img_channel):
        self.path = path
        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_channel = img_channel
        self.indexes = np.arange(len(self.list_IDs))
        
    def __len__(self):
        return int(np.floor(len(self.list_IDs)/self.batch_size))
    
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, self.img_size, self.img_size, self.img_channel))
        y = np.empty((self.batch_size, 5), dtype=int)
        for i, ID in enumerate(list_IDs_temp):
            data_file = cv2.imread(self.path+ID)
            img = cv2.resize(data_file, (self.img_size, self.img_size))
            X[i, ] = img
            y[i, ] = self.labels[i]
        X = X.astype('float32')
        X -= X.mean()
        X /= X.std()
        return X, y

## Model Architecture 

In [ ]:
model = Sequential()
model.add(Conv2D(128, input_shape=(img_size,img_size,img_channel), kernel_size=5, strides=4, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(4)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

## Training Phase

In [ ]:
train_generator = DataGenerator(path+'train_images/', train_data['image_id'], y_train, batch_size, img_size, img_channel)

In [ ]:
history = model.fit_generator(generator=train_generator,
                              epochs = 3,
                              class_weight = class_weight,
                              workers=4)

# Model Architecture

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True)

## Next Transfer learning approch to improve the accuracy of the model